In [2]:
import numpy as np
import pandas as pd

In [18]:
#df = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\around_station_small.parquet')
#df = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\2018_part_0_small_ALL_data_with_lightning.parquet')
df = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted\\around_station_2018_part_3_of_6_small.parquet')

In [19]:
print(df.shape)

(1664, 27)


In [5]:
df.head(6)

,File_Name,Cloud_Number,Area,Center_lonlat_1,Center_lonlat_2,Eccentricity,Orientation,TBBmax,TBBmean,TBBmedian,...,Minute,Second,Latitude,Longitude,Error of location,Number of discharges,Number of day,Lightning is in cloud,Lightning is in ellipse,TBB near lightning
0,20180101_0230,106,427.332532,43.244087,35.702950,0.730052,-15.513780,-32.508007,-33.519859,-33.721004,...,21.0,37.986630,35.6486,43.3225,7.5,5.0,737061.098356,0.0,1.0,-32.906994
1,20180101_0230,106,427.332532,43.244087,35.702950,0.730052,-15.513780,-32.508007,-33.519859,-33.721004,...,23.0,37.389205,35.6738,43.3872,20.3,11.0,737061.099738,0.0,1.0,-33.313000
2,20180101_0230,106,427.332532,43.244087,35.702950,0.730052,-15.513780,-32.508007,-33.519859,-33.721004,...,46.0,21.566376,35.6806,43.2386,5.8,7.0,737061.115527,1.0,1.0,-34.132000
3,20180101_0830,686,3225.034520,44.632042,35.538177,0.859746,-76.105774,-32.074001,-33.479031,-33.275997,...,6.0,4.863414,35.1790,44.5260,7.5,6.0,737061.337556,0.0,1.0,-32.870998
4,20180101_0830,686,3225.034520,44.632042,35.538177,0.859746,-76.105774,-32.074001,-33.479031,-33.275997,...,7.0,24.838277,35.2499,44.5339,13.6,6.0,737061.338482,1.0,1.0,-34.096004
5,20180101_0830,686,3225.034520,44.632042,35.538177,0.859746,-76.105774,-32.074001,-33.479031,-33.275997,...,7.0,24.838277,35.2789,44.5393,7.3,5.0,737061.338482,1.0,1.0,-35.130001


In [6]:
print(df.columns)

Index(['File_Name', 'Cloud_Number', 'Area', 'Center_lonlat_1',
       'Center_lonlat_2', 'Eccentricity', 'Orientation', 'TBBmax', 'TBBmean',
       'TBBmedian', 'TBBmin', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second', 'Latitude', 'Longitude', 'Error of location',
       'Number of discharges', 'Number of day', 'Lightning is in cloud',
       'Lightning is in ellipse', 'TBB near lightning'],
      dtype='object')


## Number of discharges per minute, by WWLLN

In [10]:
import pandas as pd

# Assuming df is already loaded
# Convert time-related columns to integer values
df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']] = df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']].astype(int)

# Create a full datetime column from these components
df['Timestamp'] = pd.to_datetime(
    df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']]
)

# Aggregate by minute by flooring the timestamp to the nearest minute
df['Minute_Timestamp'] = df['Timestamp'].dt.floor('T')  # 'T' for minute

# Count the number of discharges for each minute
minute_discharge_counts = df.groupby('Minute_Timestamp').size().reset_index(name='Discharge_Count')

# Print the table
print(minute_discharge_counts)



       Minute_Timestamp  Discharge_Count
0   2018-01-01 02:21:00                1
1   2018-01-01 02:23:00                1
2   2018-01-01 02:46:00                1
3   2018-01-01 08:06:00                1
4   2018-01-01 08:07:00                2
..                  ...              ...
357 2018-01-01 22:18:00                3
358 2018-01-01 22:26:00                1
359 2018-01-01 23:06:00                2
360 2018-01-01 23:10:00                1
361 2018-01-02 01:22:00                1

[362 rows x 2 columns]


## Number of discharges per minute, by Lightning Detector (2) (local system)

In [11]:
df_csv = pd.read_csv('F:/2024_Lightning_Analysis/Aragats_measurements/2018/2018_Lightning_Detector_1_5_km.csv')

In [13]:
df_csv.columns

Index(['Date/Time', ' CG+  Radius 5 km', ' CG-  Radius 5 km',
       ' IC+  Radius 5 km', ' IC-  Radius 5 km', 'Timestamp',
       'Minute_Timestamp'],
      dtype='object')

### Table "minute - Number of discharges" for Lightning Detector (2) csv

In [15]:
# Convert the "Date/Time" column in CSV to datetime format
df_csv['Timestamp'] = pd.to_datetime(df_csv['Date/Time'], format='%d-%b-%y %H:%M:%S.%f')
df_csv['Minute_Timestamp'] = df_csv['Timestamp'].dt.floor('T')

# Sum the four columns for discharge types to get total discharges per minute in CSV data
df_csv['Discharge_Count'] = df_csv[[' CG+  Radius 5 km', ' CG-  Radius 5 km', ' IC+  Radius 5 km', ' IC-  Radius 5 km']].sum(axis=1)

# Aggregate by minute and select relevant columns
csv_minute_discharge_counts = df_csv.groupby('Minute_Timestamp')['Discharge_Count'].sum().reset_index()

#print(parquet_minute_discharge_counts)
print("\nCSV Data (Minute - Number of Discharges):")
print(csv_minute_discharge_counts)


Parquet Data (Minute - Number of Discharges):

CSV Data (Minute - Number of Discharges):
         Minute_Timestamp  Discharge_Count
0     2018-01-01 19:51:00                0
1     2018-01-01 19:52:00                0
2     2018-01-01 21:39:00                0
3     2018-01-01 23:04:00                0
4     2018-01-02 09:58:00                0
...                   ...              ...
23758 2018-12-27 23:12:00                0
23759 2018-12-27 23:13:00                0
23760 2018-12-27 23:16:00                0
23761 2018-12-27 23:18:00                0
23762 2018-12-28 12:58:00                0

[23763 rows x 2 columns]


### Table "minute - Number of discharges" for WWLLN and Lightning Detector (2)

In [16]:
# Convert time-related columns in parquet data to integer
df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']] = df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']].astype(int)

# Create a full timestamp for the parquet data
df['Timestamp'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])
df['Minute_Timestamp'] = df['Timestamp'].dt.floor('T')  # Floor to nearest minute
parquet_minute_discharge_counts = df.groupby('Minute_Timestamp').size().reset_index(name='Discharge_Count')

In [17]:
import pandas as pd

# Assuming you already have the parquet_minute_discharge_counts DataFrame
# and csv_minute_discharge_counts from the code above.

# Ensure the Parquet data has 'Minute_Timestamp' column
parquet_minute_discharge_counts.rename(columns={'Discharge_Count': 'Parquet_Discharge_Count'}, inplace=True)
csv_minute_discharge_counts.rename(columns={'Discharge_Count': 'CSV_Discharge_Count'}, inplace=True)

# Find the time range in the Parquet data
time_min, time_max = parquet_minute_discharge_counts['Minute_Timestamp'].min(), parquet_minute_discharge_counts['Minute_Timestamp'].max()

# Filter the CSV data to match the Parquet data time range
csv_minute_discharge_counts_filtered = csv_minute_discharge_counts[
    (csv_minute_discharge_counts['Minute_Timestamp'] >= time_min) &
    (csv_minute_discharge_counts['Minute_Timestamp'] <= time_max)
]

# Merge the CSV and Parquet data on the 'Minute_Timestamp' column
combined_discharge_counts = pd.merge(
    parquet_minute_discharge_counts,
    csv_minute_discharge_counts_filtered,
    on='Minute_Timestamp',
    how='inner'
)

# Print the combined table
print("\nCombined Data (Minute - Number of Discharges from CSV - Number of Discharges from Parquet):")
print(combined_discharge_counts)



Combined Data (Minute - Number of Discharges from CSV - Number of Discharges from Parquet):
     Minute_Timestamp  Parquet_Discharge_Count  CSV_Discharge_Count
0 2018-01-01 19:51:00                       17                    0
1 2018-01-01 19:52:00                       14                    0


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load parquet data
df = pd.read_parquet('F:/2024_Lightning_Analysis/FY-2E_data/data_extracted/2018_part_0_small_ALL_data_with_lightning.parquet')

# Convert time-related columns in parquet data to integer
df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']] = df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']].astype(int)

# Create a full timestamp for the parquet data
df['Timestamp'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute', 'Second']])
df['Minute_Timestamp'] = df['Timestamp'].dt.floor('T')  # Floor to nearest minute
parquet_minute_discharge_counts = df.groupby('Minute_Timestamp').size().reset_index(name='Discharge_Count')

# Load CSV data
df_csv = pd.read_csv('your_csv_path_here.csv')

# Convert the "Date/Time" column in CSV to datetime format
df_csv['Timestamp'] = pd.to_datetime(df_csv['Date/Time'], format='%d-%b-%y %H:%M:%S.%f')
df_csv['Minute_Timestamp'] = df_csv['Timestamp'].dt.floor('T')

# Sum the four columns for discharge types to get total discharges per minute in CSV data
df_csv['Discharge_Count'] = df_csv[['CG+  Radius 5 km', 'CG-  Radius 5 km', 'IC+  Radius 5 km', 'IC-  Radius 5 km']].sum(axis=1)

# Aggregate by minute and select relevant columns
csv_minute_discharge_counts = df_csv.groupby('Minute_Timestamp')['Discharge_Count'].sum().reset_index()

# Filter CSV data to only include timestamps within the parquet data range
time_min, time_max = parquet_minute_discharge_counts['Minute_Timestamp'].min(), parquet_minute_discharge_counts['Minute_Timestamp'].max()
csv_minute_discharge_counts = csv_minute_discharge_counts[(csv_minute_discharge_counts['Minute_Timestamp'] >= time_min) & 
                                                          (csv_minute_discharge_counts['Minute_Timestamp'] <= time_max)]

# Print both tables
print("Parquet Data (Minute - Number of Discharges):")
print(parquet_minute_discharge_counts)
print("\nCSV Data (Minute - Number of Discharges):")
print(csv_minute_discharge_counts)

# Plot both data sources on the same graph
plt.figure(figsize=(12, 6))
plt.plot(parquet_minute_discharge_counts['Minute_Timestamp'], parquet_minute_discharge_counts['Discharge_Count'], label='Parquet Data', color='blue', alpha=0.7)
plt.plot(csv_minute_discharge_counts['Minute_Timestamp'], csv_minute_discharge_counts['Discharge_Count'], label='CSV Data', color='orange', alpha=0.7)

# Label the plot
plt.xlabel('Minute Timestamp')
plt.ylabel('Number of Discharges')
plt.title('Comparison of Discharges per Minute (Parquet vs. CSV Data)')
plt.legend()
plt.grid(True)

# Display the plot
plt.show()
